## Running an open-loop focus (and collimation) of the AT and LATISS
#### This notebook is used to demonstrate how to take curvature wavefront sensing (CWFS) data to calculate focus and any decentering of the hexapod. It then shows how to apply those offsets

In [ ]:
import sys
import asyncio
import logging
import numpy as np
import matplotlib.pyplot as plt
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
import time
from lsst.ts import salobj

In [ ]:
# Temporarily needed to run at summit, this will need to be commented out in the future!
import os
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [ ]:
# Creating Logging Ability
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
# Read in the class
# Note that setting remotes=False will allow one to reduce data if they so choose without affecting telescope operations
script = LatissCWFSAlign(index=1, remotes=True)
script.dataPath='/project/shared/auxTel/'

In [ ]:
# Enable Latiss (if required)
# await script.latiss.enable()

In [ ]:
#slew to target
await script.attcs.slew_object('HD 59468')

In [ ]:
# Setup LATISS to have proper configuration
script.filter='BG40'
script.grating = 'empty_1'
script.exposure_time = 30

### Take quick Image to check pointing
#### Mostly not required, therefore leaving as commented out

In [ ]:
# Check Pointing 
# await script.latiss.take_engtest(exptime=exptime, filter=script.filter, grating=script.grating)

In [ ]:
# Offset from Boresight if required (in arcseconds) - not generally required!
#await script.attcs.offset_xy(y=0,x=60) # not cumulative (sticky) offsets!

### Setup for running the script to take data and calculate offsets

In [ ]:
script.binning = 1 # takes longer to calculate but answer has higher confidence, should be able to run with binning=2

In [ ]:
# Uncomment here to run the script without taking new images
# To test a single pair of images just put image id's. Set to None to get new images without restarting the notebook.

#script.intra_visit_id = 2020031200289
#script.extra_visit_id = 2020031200290

# This angle should be set to the angle relating the instrument to the boresight, which is
# equal to "elevation - nasmyth2 rotator angle". It needs to be manually set
# as the script uses an event which won't be published.

#script.angle = 0.0 #61.7-37.9 # elev - nas2 

In [ ]:
start_time=time.time()
await script.run_cwfs()
end_time=time.time()
print('WFE fitting took {0:0.3f} seconds'.format(end_time-start_time)) # 56.7s

## Print and plot the results of the fit
#### If the coma X, Y and defocus results are below 50nm then this is acceptable and the PSF will be heavily dominated by atmospheric seeing

In [ ]:
script.calculate_results()
# plot zernikes
x = np.arange(9)+4
plt.plot(x, script.algo.zer4UpNm[:9], 'o-', label=f'{script.dz}')
xlim = plt.xlim()
plt.plot(np.arange(15), np.zeros(15)+50, 'b--')
plt.plot(np.arange(15), np.zeros(15)-50, 'b--')
plt.xlim(xlim)
plt.ylabel("Zernike coeff (nm)")
plt.xlabel("Zernike index")
plt.grid()
plt.legend()

print(script.algo.zer4UpNm[:9])

In [ ]:
# plot image and mask 
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title("defocus 0.8 - intra")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title("defocus 0.8 - extra")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 

## Take an in-focus image (not required)
#### This can be useful to verify WFE due to the optics are not dominent

In [ ]:
if False:
    # Start a logger to publish observer notes to the EFD, this is useful for finding the data later but not required
    # find it using SELECT "message" FROM "efd"."autogen"."lsst.sal.Script.logevent_logMessage" WHERE time > :dashboardTime:
    comment_log_controller = salobj.Controller("Script", index=1)
    comment_log_controller.evt_logMessage.set_put(message="Starting In-focus post CWFS data on current target")
    #comment_log_controller.evt_logMessage.set_put(message="Test: {}".format(time.time()))
    tmp = latiss.take_object(exptime=15, n=1)
    comment_log_controller.evt_logMessage.set_put(message="Finished In-focus post CWFS data current target"))

## Apply hexapod offsets
### Values are left to be input by hand to do manual "gain" corrections. See tstn-015.lsst.io for discussion on this topic
#### Defocus offsets less than 2um should not be applied
#### Coma (x,y) offsets less than 50um should not be applied

In [ ]:
# command to offset hexapod
offset = {'x':0, 'y':  0.0, 'z': 0.0}
await script.attcs.ataos.cmd_offset.set_start(**offset)

In [ ]:
# Offsets are cumulative, if you get lost you can go back to what the look-up table originally 
# planned (and probably where you started from) by uncommenting the following line
#await script.attcs.ataos.cmd_resetOffset.start()